### Just a training with pyspark running on linux

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [6]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("sparkDataframeApi")
    .getOrCreate()
)

24/02/29 16:07:12 WARN Utils: Your hostname, DESKTOP-9M9GKHV resolves to a loopback address: 127.0.1.1; using 172.28.62.164 instead (on interface eth0)
24/02/29 16:07:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 16:07:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/29 16:07:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
df = (
    spark
    .read
    .option("delimiter", ';')
    .option("header", 'true')
    .option("inferSchema", 'true')
    .option("enconding", 'ISO-8859-1')
    .csv(r"./data.test/Preços semestrais - AUTOMOTIVOS_2023.02.csv")
)

In [9]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [11]:
df_precos = (
    df.select('Estado - Sigla', 'Produto', 'Valor de Compra', 'Valor de Venda', 'Unidade de Medida')
)

In [15]:
df_precos.show(5)

+--------------+----------+---------------+--------------+-----------------+
|Estado - Sigla|   Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+----------+---------------+--------------+-----------------+
|            SP|  GASOLINA|           NULL|          4,87|       R$ / litro|
|            SP|DIESEL S10|           NULL|          4,88|       R$ / litro|
|            SP|    ETANOL|           NULL|          3,27|       R$ / litro|
|            AC|  GASOLINA|           NULL|          6,95|       R$ / litro|
|            AC|DIESEL S10|           NULL|          6,85|       R$ / litro|
+--------------+----------+---------------+--------------+-----------------+
only showing top 5 rows



In [16]:
df_precos.where ( F.col('Valor de Compra').isNotNull()).show()

+--------------+-------+---------------+--------------+-----------------+
|Estado - Sigla|Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+-------+---------------+--------------+-----------------+
+--------------+-------+---------------+--------------+-----------------+



In [20]:
df_precos = (
    df
    .select('Estado - Sigla', 'Produto', 'Valor de Venda', 'Unidade de Medida')
    .withColumn(
        "Valor de Venda",
        F.regexp_replace(F.col("Valor de Venda"), ",", ".")
        .cast("float")
    )
)
#removing 'Valor de Compra' because is a full null column
#replacing values of 'Valor de Venda' changing string to float and adapting to Brazilian writing (, for .)

In [22]:
df_precos.show(5) #its work

+--------------+----------+--------------+-----------------+
|Estado - Sigla|   Produto|Valor de Venda|Unidade de Medida|
+--------------+----------+--------------+-----------------+
|            SP|  GASOLINA|          4.87|       R$ / litro|
|            SP|DIESEL S10|          4.88|       R$ / litro|
|            SP|    ETANOL|          3.27|       R$ / litro|
|            AC|  GASOLINA|          6.95|       R$ / litro|
|            AC|DIESEL S10|          6.85|       R$ / litro|
+--------------+----------+--------------+-----------------+
only showing top 5 rows



In [29]:
df_precos_analise = (
    df_precos
    .groupBy(
        F.col('Estado - Sigla'),
        F.col('Produto'),
        F.col('Unidade de Medida')
    )
    .agg(
        F.min(F.col("Valor de Venda")).alias('minPrice'),
        F.max(F.col("Valor de Venda")).alias('maxPrice')
    )
    .withColumn(
        "dif",
        F.col('maxPrice') - F.col('minPrice')
    )
    .orderBy('dif', ascending=False)
)
# making some analisis with de dif of the maxprice and minprice of the fuel in brazil 
# and showing on a crescending order, we can se in the RS is the most differene

In [30]:
df_precos_analise.show(10)

+--------------+------------------+-----------------+--------+--------+---------+
|Estado - Sigla|           Produto|Unidade de Medida|minPrice|maxPrice|      dif|
+--------------+------------------+-----------------+--------+--------+---------+
|            RS|GASOLINA ADITIVADA|       R$ / litro|    4.89|    9.79|      4.9|
|            SP|        DIESEL S10|       R$ / litro|    4.27|     9.0|     4.73|
|            RJ|        DIESEL S10|       R$ / litro|    4.48|    8.89|4.4100003|
|            AM|        DIESEL S10|       R$ / litro|    4.79|    8.89|4.1000004|
|            BA|GASOLINA ADITIVADA|       R$ / litro|    4.89|    8.94|4.0499997|
|            SP|GASOLINA ADITIVADA|       R$ / litro|    4.69|    8.69|3.9999995|
|            PE|        DIESEL S10|       R$ / litro|    4.34|    8.19|3.8499994|
|            PR|GASOLINA ADITIVADA|       R$ / litro|    4.79|    8.49|3.6999998|
|            SP|            ETANOL|       R$ / litro|    2.69|    6.29|      3.6|
|            RO|